In [3]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [4]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-10 23:46:32--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  4.96MB/s    in 0.2s    

2022-07-10 23:46:33 (4.96 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|          review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|         US|    1797882|R3I2DHQBR577SS|B001ANOOOE|       2102612|The Naked Bee Vit...|          Beauty|          5|            0|          0|   N|                Y|          Five Stars| Love this, excell...| 2015-08-31|
|         US|   18381298|R1QNE9NQFJC2Y4|B0016J22EQ|     106393691|Alba Botanica Sun...|          Beauty|          5|

In [7]:
df.count()

5115666

In [8]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter for only columns with total_votes equal or greater than 20
total_votes_20greater_df = df.filter(col("total_votes") >= 20)
total_votes_20greater_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22393078|R2QRUE9REK8OUC|B00461F4PA|     608719013|Baby Foot Exfolia...|          Beauty|          5|           18|         23|   N|                Y|Better Than Any Pedi|First off, I'll s...| 2015-08-31|
|         US|   46053231|R2POXRW6PXHXZG|B00OCJZ0GE|     717345177|Bed Head Little T...|          Beauty|          5|    

In [9]:
total_votes_20greater_df.count()

79227

In [10]:
# Filter the data for helpful_votes/total_votes is >= 50%
helpful_votes_50plus_df = total_votes_20greater_df.filter(col("helpful_votes")/col("total_votes") >= .50)
helpful_votes_50plus_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22393078|R2QRUE9REK8OUC|B00461F4PA|     608719013|Baby Foot Exfolia...|          Beauty|          5|           18|         23|   N|                Y|Better Than Any Pedi|First off, I'll s...| 2015-08-31|
|         US|   46053231|R2POXRW6PXHXZG|B00OCJZ0GE|     717345177|Bed Head Little T...|          Beauty|          5|    

In [11]:
helpful_votes_50plus_df.count()

74760

In [12]:
# Filter for data where a review was written as part of the Vine program
vine_review_df = helpful_votes_50plus_df.filter(col("vine") == 'Y')
vine_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   47770601|R106V6GUNWRSSS|B00YAR86BY|     136411552|Philips Sonicare ...|          Beauty|          5|          857|        889|   Y|                N|Improved build, g...|According to spec...| 2015-08-27|
|         US|   52846213|R37N8KZS48W36A|B00YJJXJ06|     791197070|Maybelline New Yo...|          Beauty|          5|    

In [13]:
vine_review_df.count()

647

In [14]:
# Filter for data where a review was not part of the Vine program
not_vine_df = helpful_votes_50plus_df.filter(col("vine") == 'N')
not_vine_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22393078|R2QRUE9REK8OUC|B00461F4PA|     608719013|Baby Foot Exfolia...|          Beauty|          5|           18|         23|   N|                Y|Better Than Any Pedi|First off, I'll s...| 2015-08-31|
|         US|   46053231|R2POXRW6PXHXZG|B00OCJZ0GE|     717345177|Bed Head Little T...|          Beauty|          5|    

In [15]:
not_vine_df.count()

74113

In [16]:
# Finding total # of 5-star paid vine reviews 
paid_star_rating_df = vine_review_df.filter(col("star_rating") == 5)
paid_star_rating_df.count()

229

In [17]:
# Finding total # of 5-star unpaid reviews 
unpaid_star_rating_df = not_vine_df.filter(col("star_rating") == 5)
unpaid_star_rating_df.count()

43217

In [18]:
# Finding percentage of 5-star paid vine reviews
percent_paid = (paid_star_rating_df.count()/vine_review_df.count())*100
print("Percentage of 5-star reviews for paid Vine program was: %f" % percent_paid + "%")

Percentage of 5-star reviews for paid Vine program was: 35.394127%


In [19]:
# Finding percentage of 5-star unpaid reviews
percent_unpaid = (unpaid_star_rating_df.count()/not_vine_df.count())*100
print("Percentage of 5-star unpaid reviews was: %f" % percent_unpaid + "%")

Percentage of 5-star unpaid reviews was: 58.312307%


In [20]:
# Finding total # of 1-star paid vine reviews 
paid_1star_rating_df = vine_review_df.filter(col("star_rating") == 1)
paid_1star_rating_df.count()

37

In [21]:
# Finding total # of 1-star unpaid reviews 
unpaid_1star_rating_df = not_vine_df.filter(col("star_rating") == 1)
unpaid_1star_rating_df.count()

14506

In [24]:
# Finding percentage of 1-star paid vine reviews
percent_paid_1 = (paid_1star_rating_df.count()/vine_review_df.count())*100
print("Percentage of 1-star reviews for paid Vine program was: %f" % percent_paid_1 + "%")

Percentage of 1-star reviews for paid Vine program was: 5.718702%


In [26]:
# Finding percentage of 1-star unpaid reviews
percent_unpaid_1 = (unpaid_1star_rating_df.count()/not_vine_df.count())*100
print("Percentage of 1-star unpaid reviews was: %f" % percent_unpaid_1 + "%")

Percentage of 1-star unpaid reviews was: 19.572814%
